In [14]:
import pandas as pd
from dataretrieval import nwis

site_number = "01108000"  # Replace with your site number
start_date = "2024-01-01"          # Replace with your start date
end_date = "2024-01-31" 



nwis.get_dv(
            sites=site_number,
            parameterCd="00060",
            statCd="00003",
            start=start_date,
            end=end_date,
        )
 

ModuleNotFoundError: No module named 'dataretrieval'

In [12]:
full_url = f"{url}?{requests.compat.urlencode(params)}"
print(full_url)
print('https://nwis.waterservices.usgs.gov/nwis/iv/?sites=01108000&startDT=2017-01-01T00:00:00.000-05:00&endDT=2017-01-02T23:59:59.999-05:00&parameterCd=00060&format=rdb')

https://nwis.waterservices.usgs.gov/nwis/iv?sites=01108000&startDT=2024-01-01T00%3A00%3A00.000-05%3A00&endDT=2024-01-31T23%3A59%3A59.999-05%3A00&parameterCd=00060&format=rdb&tz=UTC
https://nwis.waterservices.usgs.gov/nwis/iv/?sites=01108000&startDT=2017-01-01T00:00:00.000-05:00&endDT=2017-01-02T23:59:59.999-05:00&parameterCd=00060&format=rdb
